# Data integration

For each sub-dataset, write (and execute) code that converts a file (using possibly an old schema) into a file that has the new, latest schema version.

Your conversion code should not modify the original files, but instead create a new file. Be sure to explain the design behind your conversion functions!

The data integration step is highly parallellizable. Therefore, your solution on this part
**must** be written in Spark

WARNING: this notebook assumes that:

- The data are in "MY_PARENT_FOLDER/data/sampled/" folder. You can run the bash script "download_metadata.sh" to download data and metadata in the correct folders to execute the jupyter notebooks.
- The data are sampled to be run on a personnal computer.

In [24]:
# Imports go here
import os
import glob
import pandas as pd
import os 
import shutil
from shutil import copyfile
os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=3g  pyspark-shell"
from pyspark.sql import SparkSession
try: 
    spark
    print("Spark application already started. Terminating existing application and starting new one")
    spark.stop()
except: 
    pass
# Create a new spark session (note, the * indicates to use all available CPU cores)
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("H600 L-Group") \
    .getOrCreate()
#When dealing with RDDs, we work the sparkContext object. See https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext
sc=spark.sparkContext
#in local mode, you will be able to access the Spark GUI at http://localhost:4040

Spark application already started. Terminating existing application and starting new one


In [13]:
#create t2 directories
try:
    os.mkdir("data/t2")
except OSError:
    print ("Creation of the directory data/t2 failed")
else:
    print ("Successfully created the directory data/t2")

list_taxi = ["yellow", "green", "fhv", "fhvhv"]
#list_taxi = ["green"]
for taxi_brand in list_taxi :
    path = "data/t2/%s" %(taxi_brand)
    # List the file from the same taxi company brand 
    try:
        os.mkdir(path)  
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)
    

Creation of the directory data/t2 failed
Creation of the directory data/t2/yellow failed
Creation of the directory data/t2/green failed
Creation of the directory data/t2/fhv failed
Creation of the directory data/t2/fhvhv failed


## 1. FHVHV files

From previous analyses we saw that header was consistent across all then fhvhv files.
We can then create a unified file.

In [45]:

#export new csv file
#fhvhv_DF.write.save(path='data/modified/fhvhv.csv', format='csv', mode='append', sep='\t')
#ça fait un dossier ac plusieurs fichiers pourris

#move files to t2 directory
source_dir= 'data/sampled/'       
for filename in glob.glob(os.path.join(source_dir,'fhvhv_*.csv')):
    shutil.copy(filename, 'data/t2/fhvhv')

print ('list of fhvhv tripdata files:')
!ls data/t2/fhvhv

print ('count of fhvhv tripdata files:')
!find data/t2/fhvhv -type f | wc -l 

list of fhvhv tripdata files:
fhvhv_tripdata_2019-02.csv  fhvhv_tripdata_2020-01.csv
fhvhv_tripdata_2019-03.csv  fhvhv_tripdata_2020-03.csv
fhvhv_tripdata_2019-04.csv  fhvhv_tripdata_2020-04.csv
fhvhv_tripdata_2019-05.csv  fhvhv_tripdata_2020-05.csv
fhvhv_tripdata_2019-06.csv  fhvhv_tripdata_2020-06.csv
count of fhvhv tripdata files:
10


## 2.FHV files

From previous analyses we saw that for fhv there are some adjustements:

In 2017 - 1 :

4 diff on a total of 5 col: ['pickup_datetime', 'dropoff_datetime', 'pulocationid', 'dolocationid']
           2/4 col add
           2/4 name change

In 2017 - 7 :

1 diff on a total of 6 col: ['sr_flag']
           1/1 col add

In 2018 - 1 :

1 diff on a total of 7 col: ['dispatching_base_number']
           1/1 col add

In 2019 - 1 :

1 diff on a total of 6 col: ['dispatching_base_number']
           1/1 col remove



In [51]:
source_dir= 'data/sampled/'       
#or filename in glob.glob(os.path.join(source_dir,'fhv_*.csv')):
#   shutil.copy(filename, 'data/t2/fhv')

#rint ('list of fhv tripdata files:')
#ls data/t2/fhv

#rint ('count of fhv tripdata files:')
#find data/t2/fhv -type f | wc -l 

list of fhvhv tripdata files:
fhv_tripdata_2015-01.csv  fhv_tripdata_2016-11.csv  fhv_tripdata_2018-09.csv
fhv_tripdata_2015-02.csv  fhv_tripdata_2016-12.csv  fhv_tripdata_2018-10.csv
fhv_tripdata_2015-03.csv  fhv_tripdata_2017-01.csv  fhv_tripdata_2018-11.csv
fhv_tripdata_2015-04.csv  fhv_tripdata_2017-02.csv  fhv_tripdata_2018-12.csv
fhv_tripdata_2015-05.csv  fhv_tripdata_2017-03.csv  fhv_tripdata_2019-01.csv
fhv_tripdata_2015-06.csv  fhv_tripdata_2017-04.csv  fhv_tripdata_2019-02.csv
fhv_tripdata_2015-07.csv  fhv_tripdata_2017-05.csv  fhv_tripdata_2019-03.csv
fhv_tripdata_2015-08.csv  fhv_tripdata_2017-06.csv  fhv_tripdata_2019-04.csv
fhv_tripdata_2015-09.csv  fhv_tripdata_2017-07.csv  fhv_tripdata_2019-05.csv
fhv_tripdata_2015-10.csv  fhv_tripdata_2017-08.csv  fhv_tripdata_2019-06.csv
fhv_tripdata_2015-11.csv  fhv_tripdata_2017-09.csv  fhv_tripdata_2019-07.csv
fhv_tripdata_2015-12.csv  fhv_tripdata_2017-10.csv  fhv_tripdata_2019-08.csv
fhv_tripdata_2016-01.csv  fhv_tripdata_2017-11

In [105]:
#read fhv files
#fhv1_DF = (spark.read
#           .option("sep", ",")
#           .option("header", True)
#           .option("inferSchema", True)
#            .csv('data/sampled/fhv_tripdata_2015-01.csv'))
#fhv1_DF.printSchema()
#fhv1_DF.show(1)
#fhv2_DF = (spark.read
#           .option("sep", ",")
#           .option("header", True)
#           .option("inferSchema", True)
#            .csv('data/sampled/fhv_tripdata_2017-01.csv'))
#fhv2_DF.printSchema()
##fhv2_DF.show(1)
#fhv3_DF = (spark.read
#           .option("sep", ",")
#           .option("header", True)
#           .option("inferSchema", True)
#            .csv('data/sampled/fhv_tripdata_2017-07.csv'))
#fhv3_DF.printSchema()
#fhv3_DF.show(1)
fhv4_DF = (spark.read
           .option("sep", ",")
           .option("header", True)
           .option("inferSchema", True)
            .csv('data/sampled/fhv_tripdata_2018-01.csv'))
fhv4_DF.printSchema()
fhv4_DF.show(1)
fhv5_DF = (spark.read
           .option("sep", ",")
           .option("header", True)
           .option("inferSchema", True)
            .csv('data/sampled/fhv_tripdata_2019-01.csv'))
fhv5_DF.printSchema()
fhv5_DF.show(1)

#avant 01/2015 il faut ajouter |dropoff_datetime||DOLocationID|SR_Flag
# modifier Pickup_date en pickup_datetime, locationID en PULocationID, Dispatching_base_num en dispatching_base_num


root
 |-- Pickup_DateTime: timestamp (nullable = true)
 |-- DropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: integer (nullable = true)
 |-- Dispatching_base_number: string (nullable = true)
 |-- Dispatching_base_num: string (nullable = true)

+-------------------+-------------------+------------+------------+-------+-----------------------+--------------------+
|    Pickup_DateTime|   DropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Dispatching_base_number|Dispatching_base_num|
+-------------------+-------------------+------------+------------+-------+-----------------------+--------------------+
|2018-01-20 20:07:22|2018-01-20 20:16:51|          62|          17|      1|                 B02510|                null|
+-------------------+-------------------+------------+------------+-------+-----------------------+--------------------+
only showing top 1 row

root
 |-- dispatching_base_

In [106]:
from pyspark.sql.functions import col, lit

#file_name='fhv_tripdata_2015-01.csv'
#source_dir= 'data/sampled/' 
#dest_dir='data/t2/fhv'

#1.modifing schema as needed
fhv_201501 = fhv1_DF.withColumn("dropoff_datetime",lit('null'))\
                    .withColumn("DOLocationID",lit('null'))\
                    .withColumn("SR_Flag",lit('null'))\
                    .select(
                        col("Dispatching_base_num").alias("dispatching_base_num"),
                        col("Pickup_date").alias("pickup_datetime"),
                        "dropoff_datetime",
                        col("locationID").alias("PULocationID"),
                        "DOLocationID",
                        "SR_Flag")
fhv_201501.show(1)
#saving new file                                     
#export new csv file
#fhv_201501.write.save(path='data/t2/fhv/fhv_tripdata_2015-01.csv', format='csv', mode='append', sep=',')
#print ('list of fhv tripdata files:')
#!ls data/t2/fhv

#2.modifing schema as needed
fhv_201701 = fhv2_DF.withColumn("DOLocationID",lit('null'))\
                    .withColumn("SR_Flag",lit('null'))\
                    .select(
                        col("Dispatching_base_num").alias("dispatching_base_num"),
                        col("Pickup_DateTime").alias("pickup_datetime"),
                        col("Dropoff_datetime").alias("dropoff_datetime"),
                        "PULocationID",
                        "DOLocationID",
                        "SR_Flag")
fhv_201701.show(1)
#3.modifing schema as needed
fhv_201707 = fhv3_DF.select(
                        col("Dispatching_base_num").alias("dispatching_base_num"),
                        col("Pickup_DateTime").alias("pickup_datetime"),
                        col("Dropoff_datetime").alias("dropoff_datetime"),
                        "PULocationID",
                        "DOLocationID",
                        "SR_Flag")
fhv_201707.show(1)
#2.modifing schema as needed
fhv_201801 =  fhv4_DF.select(
                        col("Dispatching_base_num").alias("dispatching_base_num"),
                        col("Pickup_DateTime").alias("pickup_datetime"),
                        col("Dropoff_datetime").alias("dropoff_datetime"),
                        "PULocationID",
                        "DOLocationID",
                        "SR_Flag")
fhv_201801.show(1)

+--------------------+-------------------+----------------+------------+------------+-------+
|dispatching_base_num|    pickup_datetime|dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+--------------------+-------------------+----------------+------------+------------+-------+
|              B02764|2015-01-09 23:28:06|            null|         186|        null|   null|
+--------------------+-------------------+----------------+------------+------------+-------+
only showing top 1 row

+--------------------+-------------------+----------------+------------+------------+-------+
|dispatching_base_num|    pickup_datetime|dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+--------------------+-------------------+----------------+------------+------------+-------+
|              B02914|2017-01-19 22:09:39|            null|        null|        null|   null|
+--------------------+-------------------+----------------+------------+------------+-------+
only showing top 1 row

+-----------

## 3.Green files

For green there are 76 files:

In 2015 - 1 :

   1 diff on a total of 21 col: ['improvement_surcharge']
           1/1 col add

In 2016 - 7 :

   2 diff on a total of 19 col: ['pulocationid', 'dolocationid']
           2/2 col remove

In 2019 - 1 :

   1 diff on a total of 20 col: ['congestion_surcharge']
           1/1 col add



##.Yellow files

For yellow there are 131 files:

 In 2010 - 1 :
 
   12 diff on a total of 18 col: ['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'pickup_longitude', 'pickup_latitude', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'fare_amount', 'tip_amount', 'tolls_amount', 'total_amount']
         12/12 column name have changed:
         
 In 2015 - 1 :
 
   6 diff on a total of 19 col: ['vendorid', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'ratecodeid', 'extra', 'improvement_surcharge']
         1/6 col add
         5/6 name change
         
 In 2016 - 7 :
 
   2 diff on a total of 17 col: ['pulocationid', 'dolocationid']
         2/2 col remove
         
 In 2019 - 1 :
 
   1 diff on a total of 18 col: ['congestion_surcharge']
         1/1 col add